In [1]:
from src.retrievers import MilvusMultiVectorRetriever

from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from pymilvus import connections
import os
from dotenv import load_dotenv


/home/damir/.cache/pypoetry/virtualenvs/huberman-rag-fdQcU6Ve-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
host = "127.0.0.1"  # Milvus server host
port = "19530"  # Milvus server port

# Connect to Milvus server
connections.connect(host=host, port=port)


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [7]:
# Template

"""If you do not have the answer in the CONTEXT write "I don't know".
Do not make things up. Do not fabricate facts. """
RAG_TEMPLATE = """\
You are a Huberman Lab podcast listener. 
You will be provided a CONTEXT of the podcasts and a QUESTION.
Based on the CONTEXT answer the QUESTION.


Context:
{context}

Query:
{question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

# Setup a chat model
chat_model = ChatOpenAI()
# Setup a retriever
retriever = MilvusMultiVectorRetriever()

# Setup a chain
multivector_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [4]:
multivector_chain.invoke({"question" : "How much to stretch to improve flexibility?"})["response"].content

'The recommended duration for static stretching to improve flexibility is 30 seconds per set. Holding a stretch for more than 30 seconds did not result in additional benefits according to the information provided.'

In [14]:
multivector_chain.invoke({"question" : "How does caffeine reinforce?"})["response"].content

'Caffeine reinforces by increasing the probability that you will return to and engage in a certain activity or consume a certain beverage or food. It also increases dopamine and acetylcholine, which are both neuromodulators in the forebrain, helping to improve our ability to think and modify our strategies for different social, mental, and physical demands. Additionally, caffeine acts as an antagonist to adenosine, offsetting sleepiness.'

In [15]:
multivector_chain.invoke({"question" : "Which compounds does coffee have?"})["response"].content

"I don't know."

In [16]:
multivector_chain.invoke({"question": "How to be a better husband?"})

{'response': AIMessage(content="I don't know.", response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 3756, 'total_tokens': 3761}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, id='run-53e8d330-e732-48be-86d5-8aeaaf934d9f-0'),
 'context': [[Document(page_content="But there's an additional one that I find particularly interesting because it addresses and indeed answers one of the most common questions that I receive all the time, which is how can I get more motivated? Not how I, Andrew can get more motivated, although of course I asked myself that question from time to time.\nAlthough admittedly, most of the time I'm wondering how I'm just going to get everything that I need to get done, done.\nBut I often get the question, how can I feel more motivated? Or what can I do to sustain my motivation over time? And we hear a lot of different strategies about how to do that.\nWe hear about the quote unqu

In [8]:
res = multivector_chain.invoke({"question" : "How much caffeine a day is the maximum limit?"})

In [9]:
res

{'response': AIMessage(content='The maximum limit of caffeine intake per day mentioned in the context is 600 milligrams per day.', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 2922, 'total_tokens': 2942}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-2c17606e-9ace-4fd4-a90e-6edd613ffd03-0'),
 'context': [[Document(page_content="We'll talk about the use of caffeine every other day.\nI know for you habitual caffeine drinkers, including myself, just the simple mention of that probably sounds aversive, but there is actually great utility to using caffeine every other day as opposed to every day.\nBut just keep in mind that some people will drink caffeine and not get much of a lift from it at all.\nOther people will drink caffeine and they will feel extremely anxious, even at dosages far lower than that one to three milligrams per kilogram of body weight range that I described a momen